<a href="https://colab.research.google.com/github/tyri0n11/Fraud-Detection-Kaggle/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Fraud Detection") \
    .getOrCreate()

In [6]:
df = spark.read.csv("Final Transaction.csv", header=True, inferSchema=True)
df.show(5)
df.dropna()
df.printSchema()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/Final Transaction.csv. SQLSTATE: 42K03

In [ ]:
df_clean = df.drop('_c0','TRANSACTION_ID', 'TX_FRAUD_SCENARIO')
df_clean.show(5)

In [ ]:
from pyspark.sql.functions import col

df_clean = df_clean.filter(col("TX_FRAUD").isNotNull())


In [ ]:
from pyspark.sql.functions import expr, hour, dayofweek

df_clean = df_clean.withColumn(
    "TX_DATETIME",
    expr("try_to_timestamp(TX_DATETIME, 'yyyy-MM-dd HH:mm:ss')")
)

df_clean = df_clean.withColumn("TX_HOUR", hour("TX_DATETIME")) \
                   .withColumn("TX_DAYOFWEEK", dayofweek("TX_DATETIME")) \
                   .drop("TX_DATETIME")



df_clean.show(5)
df_clean.printSchema()

## Data Exploration Chi Tiết

Phân tích dataset để hiểu rõ hơn về cấu trúc, phân phối và các đặc điểm của dữ liệu.


In [ ]:
# ============================================
# 1. Thống kê mô tả (Descriptive Statistics)
# ============================================
print("=" * 60)
print("THỐNG KÊ MÔ TẢ CHO CÁC FEATURES")
print("=" * 60)

# Thống kê cho numerical features
numerical_cols = ["TX_AMOUNT", "TX_TIME_SECONDS", "TX_TIME_DAYS", "TX_HOUR", "TX_DAYOFWEEK"]
df_clean.select(numerical_cols).describe().show()

# Thống kê cho categorical features
print("\n" + "=" * 60)
print("THỐNG KÊ CHO CATEGORICAL FEATURES")
print("=" * 60)
print("\nSố lượng unique customers:", df_clean.select("CUSTOMER_ID").distinct().count())
print("Số lượng unique terminals:", df_clean.select("TERMINAL_ID").distinct().count())


In [ ]:
# ============================================
# 2. Phân tích Missing Values
# ============================================
print("=" * 60)
print("PHÂN TÍCH MISSING VALUES")
print("=" * 60)

from pyspark.sql.functions import col, sum as spark_sum, count

# Đếm missing values cho từng cột
total_rows = df_clean.count()
print(f"\nTổng số rows: {total_rows:,}")

missing_stats = []
for col_name in df_clean.columns:
    missing_count = df_clean.filter(col(col_name).isNull()).count()
    missing_pct = (missing_count / total_rows) * 100 if total_rows > 0 else 0
    missing_stats.append((col_name, missing_count, missing_pct))
    if missing_count > 0:
        print(f"{col_name}: {missing_count:,} ({missing_pct:.2f}%)")

if all(missing_count == 0 for _, missing_count, _ in missing_stats):
    print("\n✓ Không có missing values trong dataset!")


In [ ]:
# ============================================
# 3. Phân tích Class Distribution
# ============================================
print("=" * 60)
print("PHÂN TÍCH CLASS DISTRIBUTION")
print("=" * 60)

from pyspark.sql.functions import count as spark_count

class_dist = df_clean.groupBy("TX_FRAUD").agg(
    spark_count("*").alias("count")
).orderBy("TX_FRAUD")

class_dist.show()

# Tính tỷ lệ
total = df_clean.count()
fraud_count = df_clean.filter(col("TX_FRAUD") == 1).count()
normal_count = df_clean.filter(col("TX_FRAUD") == 0).count()

print(f"\nTổng số giao dịch: {total:,}")
print(f"Giao dịch bình thường (0): {normal_count:,} ({normal_count/total*100:.2f}%)")
print(f"Giao dịch gian lận (1): {fraud_count:,} ({fraud_count/total*100:.2f}%)")
print(f"\nTỷ lệ imbalance: {normal_count/fraud_count:.2f}:1 (Normal:Fraud)")


In [ ]:
# ============================================
# 4. Phân tích Correlation (sử dụng Pandas cho visualization)
# ============================================
print("=" * 60)
print("PHÂN TÍCH CORRELATION")
print("=" * 60)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Convert sang Pandas để tính correlation
pdf_for_corr = df_clean.select(
    "TX_AMOUNT", "TX_TIME_SECONDS", "TX_TIME_DAYS", 
    "TX_HOUR", "TX_DAYOFWEEK", "TX_FRAUD"
).toPandas()

correlation_matrix = pdf_for_corr.corr()

# Vẽ correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title("Correlation Matrix của các Features")
plt.tight_layout()
plt.show()

print("\nCorrelation với TX_FRAUD:")
corr_with_fraud = correlation_matrix['TX_FRAUD'].sort_values(ascending=False)
print(corr_with_fraud)


In [ ]:
# ============================================
# 5. Phân tích Distribution của Features theo Fraud Class
# ============================================
print("=" * 60)
print("PHÂN TÍCH DISTRIBUTION CỦA TX_AMOUNT")
print("=" * 60)

# So sánh distribution của TX_AMOUNT giữa fraud và non-fraud
pdf_amount = df_clean.select("TX_AMOUNT", "TX_FRAUD").toPandas()

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
fraud_amounts = pdf_amount[pdf_amount['TX_FRAUD'] == 1]['TX_AMOUNT']
normal_amounts = pdf_amount[pdf_amount['TX_FRAUD'] == 0]['TX_AMOUNT']

axes[0].hist(normal_amounts, bins=50, alpha=0.7, label='Normal', color='blue', density=True)
axes[0].hist(fraud_amounts, bins=50, alpha=0.7, label='Fraud', color='red', density=True)
axes[0].set_xlabel('TX_AMOUNT')
axes[0].set_ylabel('Density')
axes[0].set_title('Distribution of TX_AMOUNT by Class')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
data_for_box = [normal_amounts, fraud_amounts]
axes[1].boxplot(data_for_box, labels=['Normal', 'Fraud'])
axes[1].set_ylabel('TX_AMOUNT')
axes[1].set_title('Box Plot of TX_AMOUNT by Class')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Thống kê
print("\nThống kê TX_AMOUNT theo class:")
print(f"Normal - Mean: {normal_amounts.mean():.2f}, Median: {normal_amounts.median():.2f}, Std: {normal_amounts.std():.2f}")
print(f"Fraud  - Mean: {fraud_amounts.mean():.2f}, Median: {fraud_amounts.median():.2f}, Std: {fraud_amounts.std():.2f}")


## Feature Engineering Nâng Cao

Tạo các features bổ sung từ dữ liệu giao dịch để cải thiện khả năng dự đoán của model.


In [ ]:
# ============================================
# Feature Engineering: Customer-level Statistics
# ============================================
from pyspark.sql import Window
from pyspark.sql.functions import count as spark_count, avg, sum as spark_sum, stddev, max as spark_max, min as spark_min

# Window function để tính statistics theo customer
customer_window = Window.partitionBy("CUSTOMER_ID").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Tính các features theo customer
df_with_customer_features = df_clean.withColumn(
    "CUSTOMER_TX_COUNT", 
    spark_count("*").over(customer_window)
).withColumn(
    "CUSTOMER_AVG_AMOUNT",
    avg("TX_AMOUNT").over(customer_window)
).withColumn(
    "CUSTOMER_TOTAL_AMOUNT",
    spark_sum("TX_AMOUNT").over(customer_window)
).withColumn(
    "CUSTOMER_STD_AMOUNT",
    stddev("TX_AMOUNT").over(customer_window)
)

# Tính số lượng giao dịch fraud trước đó của customer
fraud_window = Window.partitionBy("CUSTOMER_ID").rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)
df_with_customer_features = df_with_customer_features.withColumn(
    "CUSTOMER_PREV_FRAUD_COUNT",
    spark_sum((col("TX_FRAUD") == 1).cast("int")).over(fraud_window)
)

print("Customer-level features đã được tạo:")
df_with_customer_features.select(
    "CUSTOMER_ID", "TX_AMOUNT", "TX_FRAUD",
    "CUSTOMER_TX_COUNT", "CUSTOMER_AVG_AMOUNT", 
    "CUSTOMER_TOTAL_AMOUNT", "CUSTOMER_STD_AMOUNT",
    "CUSTOMER_PREV_FRAUD_COUNT"
).show(10)


In [ ]:
# ============================================
# Feature Engineering: Terminal-level Statistics
# ============================================
# Window function để tính statistics theo terminal
terminal_window = Window.partitionBy("TERMINAL_ID").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_with_terminal_features = df_with_customer_features.withColumn(
    "TERMINAL_TX_COUNT",
    spark_count("*").over(terminal_window)
).withColumn(
    "TERMINAL_AVG_AMOUNT",
    avg("TX_AMOUNT").over(terminal_window)
).withColumn(
    "TERMINAL_FRAUD_RATE",
    avg((col("TX_FRAUD") == 1).cast("double")).over(terminal_window)
)

print("Terminal-level features đã được tạo:")
df_with_terminal_features.select(
    "TERMINAL_ID", "TX_AMOUNT", "TX_FRAUD",
    "TERMINAL_TX_COUNT", "TERMINAL_AVG_AMOUNT", "TERMINAL_FRAUD_RATE"
).show(10)


In [ ]:
# ============================================
# Feature Engineering: Time-based Features
# ============================================
from pyspark.sql.functions import when, log

# Tạo các time-based features
df_final_features = df_with_terminal_features.withColumn(
    "IS_WEEKEND",
    when((col("TX_DAYOFWEEK") == 1) | (col("TX_DAYOFWEEK") == 7), 1).otherwise(0)
).withColumn(
    "IS_NIGHT",
    when((col("TX_HOUR") >= 22) | (col("TX_HOUR") <= 6), 1).otherwise(0)
).withColumn(
    "IS_BUSINESS_HOURS",
    when((col("TX_HOUR") >= 9) & (col("TX_HOUR") <= 17), 1).otherwise(0)
).withColumn(
    "TX_AMOUNT_LOG",
    log(col("TX_AMOUNT") + 1)  # Log transform để giảm skewness
)

print("Time-based và transformed features đã được tạo:")
df_final_features.select(
    "TX_HOUR", "TX_DAYOFWEEK", "TX_AMOUNT", "TX_AMOUNT_LOG",
    "IS_WEEKEND", "IS_NIGHT", "IS_BUSINESS_HOURS"
).show(10)


## Feature Scaling và Preprocessing

Chuẩn hóa các numerical features để cải thiện hiệu suất của các thuật toán machine learning.


In [ ]:
# ============================================
# Feature Scaling với StandardScaler
# ============================================
from pyspark.ml.feature import StandardScaler, VectorAssembler

# Cập nhật danh sách features để bao gồm các features mới
feature_cols = [
    "CUSTOMER_ID",
    "TERMINAL_ID",
    "TX_AMOUNT",
    "TX_TIME_SECONDS",
    "TX_TIME_DAYS",
    "TX_HOUR",
    "TX_DAYOFWEEK",
    "CUSTOMER_TX_COUNT",
    "CUSTOMER_AVG_AMOUNT",
    "CUSTOMER_TOTAL_AMOUNT",
    "CUSTOMER_STD_AMOUNT",
    "CUSTOMER_PREV_FRAUD_COUNT",
    "TERMINAL_TX_COUNT",
    "TERMINAL_AVG_AMOUNT",
    "TERMINAL_FRAUD_RATE",
    "IS_WEEKEND",
    "IS_NIGHT",
    "IS_BUSINESS_HOURS",
    "TX_AMOUNT_LOG"
]

# Tạo vector assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw",
    handleInvalid="keep"
)

# Transform data
df_assembled = assembler.transform(df_final_features)

# Apply StandardScaler
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

# Tạo DataFrame cho ML
df_ml = df_scaled.select("features", "TX_FRAUD")

print("Features đã được scaled:")
print(f"Số lượng features: {len(feature_cols)}")
df_ml.show(3, truncate=False)


## Cải Thiện Model Evaluation - Đầy Đủ Metrics

Tạo hàm để tính toán tất cả các metrics quan trọng cho classification.


In [ ]:
# ============================================
# Hàm tính toán đầy đủ metrics
# ============================================
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from sklearn.metrics import precision_recall_curve, auc as sklearn_auc
import pandas as pd

def calculate_all_metrics(predictions, label_col="TX_FRAUD", prediction_col="prediction", prob_col="probability"):
    """
    Tính toán tất cả các metrics cho binary classification
    """
    # Binary Classification Metrics
    auc_evaluator = BinaryClassificationEvaluator(
        labelCol=label_col,
        metricName="areaUnderROC"
    )
    auc = auc_evaluator.evaluate(predictions)
    
    # Multiclass Classification Metrics
    evaluator = MulticlassClassificationEvaluator(
        labelCol=label_col,
        predictionCol=prediction_col
    )
    
    accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
    precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
    f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
    
    # Tính Precision, Recall, F1 cho từng class
    pdf = predictions.select(label_col, prediction_col, prob_col).toPandas()
    y_true = pdf[label_col].values
    y_pred = pdf[prediction_col].values
    
    # Confusion matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Metrics cho class 1 (Fraud)
    if len(cm) == 2:
        tn, fp, fn, tp = cm.ravel()
        precision_class1 = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall_class1 = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_class1 = 2 * (precision_class1 * recall_class1) / (precision_class1 + recall_class1) if (precision_class1 + recall_class1) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else:
        precision_class1 = recall_class1 = f1_class1 = specificity = 0
    
    # Precision-Recall AUC
    y_proba = pdf[prob_col].apply(lambda x: x[1] if len(x) > 1 else x[0]).values
    pr_precision, pr_recall, _ = precision_recall_curve(y_true, y_proba)
    pr_auc = sklearn_auc(pr_recall, pr_precision)
    
    return {
        "AUC-ROC": auc,
        "AUC-PR": pr_auc,
        "Accuracy": accuracy,
        "Precision (Weighted)": precision,
        "Recall (Weighted)": recall,
        "F1-Score (Weighted)": f1,
        "Precision (Fraud Class)": precision_class1,
        "Recall (Fraud Class)": recall_class1,
        "F1-Score (Fraud Class)": f1_class1,
        "Specificity": specificity,
        "Confusion Matrix": cm
    }

def print_metrics(metrics, model_name):
    """In metrics một cách đẹp mắt"""
    print(f"\n{'='*60}")
    print(f"METRICS CHO {model_name}")
    print(f"{'='*60}")
    print(f"AUC-ROC:              {metrics['AUC-ROC']:.4f}")
    print(f"AUC-PR:                {metrics['AUC-PR']:.4f}")
    print(f"Accuracy:              {metrics['Accuracy']:.4f}")
    print(f"Precision (Weighted):  {metrics['Precision (Weighted)']:.4f}")
    print(f"Recall (Weighted):     {metrics['Recall (Weighted)']:.4f}")
    print(f"F1-Score (Weighted):   {metrics['F1-Score (Weighted)']:.4f}")
    print(f"\n--- Metrics cho Fraud Class (Class 1) ---")
    print(f"Precision:             {metrics['Precision (Fraud Class)']:.4f}")
    print(f"Recall:                {metrics['Recall (Fraud Class)']:.4f}")
    print(f"F1-Score:              {metrics['F1-Score (Fraud Class)']:.4f}")
    print(f"Specificity:           {metrics['Specificity']:.4f}")
    print(f"\nConfusion Matrix:")
    print(metrics['Confusion Matrix'])

print("Hàm calculate_all_metrics đã được định nghĩa!")


## Train/Test Split

Chia dataset thành training và testing sets.


In [ ]:
# Train/Test Split với df_ml đã được scaled
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

print("Train:", train_df.count())
print("Test :", test_df.count())


## Baseline Models với Đầy Đủ Metrics

Train và evaluate baseline models với tất cả metrics.


In [ ]:
# ============================================
# Baseline Logistic Regression
# ============================================
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="TX_FRAUD",
    maxIter=20
)

lr_model = lr.fit(train_df)
lr_pred = lr_model.transform(test_df)

# Tính đầy đủ metrics
lr_metrics = calculate_all_metrics(lr_pred, label_col="TX_FRAUD")
print_metrics(lr_metrics, "Logistic Regression (Baseline)")


In [ ]:
# ============================================
# Baseline Random Forest
# ============================================
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="TX_FRAUD",
    seed=42,
    numTrees=100
)

rf_model = rf.fit(train_df)
rf_pred = rf_model.transform(test_df)

# Tính đầy đủ metrics
rf_metrics = calculate_all_metrics(rf_pred, label_col="TX_FRAUD")
print_metrics(rf_metrics, "Random Forest (Baseline)")


## Hyperparameter Tuning với Grid Search

Sử dụng Grid Search để tìm hyperparameters tối ưu cho các models.


In [ ]:
# ============================================
# Hyperparameter Tuning cho Logistic Regression
# ============================================
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import time

print("=" * 60)
print("HYPERPARAMETER TUNING CHO LOGISTIC REGRESSION")
print("=" * 60)

# Tạo parameter grid
lr_param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Evaluator
evaluator = BinaryClassificationEvaluator(
    labelCol="TX_FRAUD",
    metricName="areaUnderROC"
)

# Cross Validator
lr_cv = CrossValidator(
    estimator=lr,
    estimatorParamMaps=lr_param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

# Fit model với cross-validation
print("\nBắt đầu Grid Search cho Logistic Regression...")
start_time = time.time()
lr_cv_model = lr_cv.fit(train_df)
end_time = time.time()

print(f"Hoàn thành trong {end_time - start_time:.2f} giây")

# Best model
lr_best_model = lr_cv_model.bestModel
print(f"\nBest parameters:")
print(f"  regParam: {lr_best_model.getRegParam()}")
print(f"  elasticNetParam: {lr_best_model.getElasticNetParam()}")

# Evaluate best model
lr_tuned_pred = lr_best_model.transform(test_df)
lr_tuned_metrics = calculate_all_metrics(lr_tuned_pred, label_col="TX_FRAUD")
print_metrics(lr_tuned_metrics, "Logistic Regression (Tuned)")


In [ ]:
# ============================================
# Hyperparameter Tuning cho Random Forest
# ============================================
print("=" * 60)
print("HYPERPARAMETER TUNING CHO RANDOM FOREST")
print("=" * 60)

# Tạo parameter grid cho Random Forest
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100, 200]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .addGrid(rf.maxBins, [32, 64]) \
    .build()

# Cross Validator cho Random Forest
rf_cv = CrossValidator(
    estimator=rf,
    estimatorParamMaps=rf_param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

# Fit model với cross-validation
print("\nBắt đầu Grid Search cho Random Forest...")
print("(Lưu ý: Quá trình này có thể mất nhiều thời gian)")
start_time = time.time()
rf_cv_model = rf_cv.fit(train_df)
end_time = time.time()

print(f"Hoàn thành trong {end_time - start_time:.2f} giây")

# Best model
rf_best_model = rf_cv_model.bestModel
print(f"\nBest parameters:")
print(f"  numTrees: {rf_best_model.getNumTrees}")
print(f"  maxDepth: {rf_best_model.getMaxDepth()}")
print(f"  maxBins: {rf_best_model.getMaxBins()}")

# Evaluate best model
rf_tuned_pred = rf_best_model.transform(test_df)
rf_tuned_metrics = calculate_all_metrics(rf_tuned_pred, label_col="TX_FRAUD")
print_metrics(rf_tuned_metrics, "Random Forest (Tuned)")


## Feature Importance Analysis

Phân tích tầm quan trọng của các features để hiểu model tốt hơn.


In [ ]:
# ============================================
# Feature Importance từ Random Forest
# ============================================
import numpy as np

# Lấy feature importance từ Random Forest model
rf_feature_importance = rf_best_model.featureImportances.toArray()

# Tạo DataFrame với feature names và importance
feature_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_feature_importance
}).sort_values('Importance', ascending=False)

print("=" * 60)
print("FEATURE IMPORTANCE (Random Forest)")
print("=" * 60)
print(feature_importance_df.to_string(index=False))

# Visualization
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(15)
plt.barh(range(len(top_features)), top_features['Importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importance (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# ============================================
# Coefficients từ Logistic Regression
# ============================================
lr_coefficients = lr_best_model.coefficients.toArray()

# Tạo DataFrame với feature names và coefficients
coefficients_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': lr_coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

print("=" * 60)
print("FEATURE COEFFICIENTS (Logistic Regression)")
print("=" * 60)
print(coefficients_df.to_string(index=False))

# Visualization
plt.figure(figsize=(12, 8))
top_coef = coefficients_df.head(15)
colors = ['red' if x < 0 else 'green' for x in top_coef['Coefficient']]
plt.barh(range(len(top_coef)), top_coef['Coefficient'], color=colors)
plt.yticks(range(len(top_coef)), top_coef['Feature'])
plt.xlabel('Coefficient Value')
plt.title('Top 15 Feature Coefficients (Logistic Regression)')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


## Precision-Recall Curves

Vẽ Precision-Recall curves cho tất cả models để so sánh hiệu suất trên imbalanced data.


In [ ]:
# ============================================
# Precision-Recall Curves cho tất cả models
# ============================================
from sklearn.metrics import precision_recall_curve

def get_pr_data(spark_df, label_col="TX_FRAUD", prob_col="probability"):
    """Lấy data cho Precision-Recall curve"""
    pdf = spark_df.select(label_col, prob_col).toPandas()
    y_true = pdf[label_col].values
    y_proba = pdf[prob_col].apply(lambda x: x[1] if len(x) > 1 else x[0]).values
    return y_true, y_proba

# Lấy data cho các models
lr_y_true, lr_y_proba = get_pr_data(lr_pred)
rf_y_true, rf_y_proba = get_pr_data(rf_pred)
lr_tuned_y_true, lr_tuned_y_proba = get_pr_data(lr_tuned_pred)
rf_tuned_y_true, rf_tuned_y_proba = get_pr_data(rf_tuned_pred)

# Tính Precision-Recall curves
lr_precision, lr_recall, _ = precision_recall_curve(lr_y_true, lr_y_proba)
rf_precision, rf_recall, _ = precision_recall_curve(rf_y_true, rf_y_proba)
lr_tuned_precision, lr_tuned_recall, _ = precision_recall_curve(lr_tuned_y_true, lr_tuned_y_proba)
rf_tuned_precision, rf_tuned_recall, _ = precision_recall_curve(rf_tuned_y_true, rf_tuned_y_proba)

# Tính AUC-PR
lr_pr_auc = sklearn_auc(lr_recall, lr_precision)
rf_pr_auc = sklearn_auc(rf_recall, rf_precision)
lr_tuned_pr_auc = sklearn_auc(lr_tuned_recall, lr_tuned_precision)
rf_tuned_pr_auc = sklearn_auc(rf_tuned_recall, rf_tuned_precision)

# Vẽ Precision-Recall curves
plt.figure(figsize=(10, 8))
plt.plot(lr_recall, lr_precision, color='blue', lw=2, 
         label=f'LR Baseline (AUC-PR = {lr_pr_auc:.3f})')
plt.plot(rf_recall, rf_precision, color='green', lw=2, 
         label=f'RF Baseline (AUC-PR = {rf_pr_auc:.3f})')
plt.plot(lr_tuned_recall, lr_tuned_precision, color='red', lw=2, 
         label=f'LR Tuned (AUC-PR = {lr_tuned_pr_auc:.3f})')
plt.plot(rf_tuned_recall, rf_tuned_precision, color='purple', lw=2, 
         label=f'RF Tuned (AUC-PR = {rf_tuned_pr_auc:.3f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves Comparison')
plt.legend(loc='lower left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## Spark Performance Optimization

Tối ưu hóa cấu hình Spark để cải thiện hiệu suất.


In [ ]:
# ============================================
# Spark Performance Optimization
# ============================================
print("=" * 60)
print("SPARK CONFIGURATION")
print("=" * 60)

# Hiển thị current Spark config
spark_conf = spark.sparkContext.getConf()
print(f"\nSpark App Name: {spark_conf.get('spark.app.name')}")
print(f"Spark Master: {spark_conf.get('spark.master', 'local[*]')}")

# Cache DataFrames để tăng tốc độ
print("\nCaching training và test DataFrames...")
train_df.cache()
test_df.cache()

# Kiểm tra số partitions
print(f"\nSố partitions của train_df: {train_df.rdd.getNumPartitions()}")
print(f"Số partitions của test_df: {test_df.rdd.getNumPartitions()}")

# Đếm để trigger caching
print("\nĐang cache DataFrames (lần đầu sẽ chậm)...")
train_count = train_df.count()
test_count = test_df.count()
print(f"Train count: {train_count:,}")
print(f"Test count: {test_count:,}")
print("\n✓ DataFrames đã được cache. Các operations tiếp theo sẽ nhanh hơn.")


## So Sánh Tổng Hợp Tất Cả Models

Tạo bảng so sánh tổng hợp cho tất cả models.


In [ ]:
# ============================================
# So sánh tổng hợp tất cả models
# ============================================
print("=" * 60)
print("SO SÁNH TỔNG HỢP TẤT CẢ MODELS")
print("=" * 60)

# Tạo comparison DataFrame
comparison_data = {
    'Model': [
        'LR Baseline',
        'RF Baseline',
        'LR Tuned',
        'RF Tuned'
    ],
    'AUC-ROC': [
        lr_metrics['AUC-ROC'],
        rf_metrics['AUC-ROC'],
        lr_tuned_metrics['AUC-ROC'],
        rf_tuned_metrics['AUC-ROC']
    ],
    'AUC-PR': [
        lr_metrics['AUC-PR'],
        rf_metrics['AUC-PR'],
        lr_tuned_metrics['AUC-PR'],
        rf_tuned_metrics['AUC-PR']
    ],
    'Accuracy': [
        lr_metrics['Accuracy'],
        rf_metrics['Accuracy'],
        lr_tuned_metrics['Accuracy'],
        rf_tuned_metrics['Accuracy']
    ],
    'Precision (Fraud)': [
        lr_metrics['Precision (Fraud Class)'],
        rf_metrics['Precision (Fraud Class)'],
        lr_tuned_metrics['Precision (Fraud Class)'],
        rf_tuned_metrics['Precision (Fraud Class)']
    ],
    'Recall (Fraud)': [
        lr_metrics['Recall (Fraud Class)'],
        rf_metrics['Recall (Fraud Class)'],
        lr_tuned_metrics['Recall (Fraud Class)'],
        rf_tuned_metrics['Recall (Fraud Class)']
    ],
    'F1-Score (Fraud)': [
        lr_metrics['F1-Score (Fraud Class)'],
        rf_metrics['F1-Score (Fraud Class)'],
        lr_tuned_metrics['F1-Score (Fraud Class)'],
        rf_tuned_metrics['F1-Score (Fraud Class)']
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# AUC-ROC và AUC-PR
axes[0, 0].bar(comparison_df['Model'], comparison_df['AUC-ROC'], color='steelblue', alpha=0.7)
axes[0, 0].set_ylabel('AUC-ROC')
axes[0, 0].set_title('AUC-ROC Comparison')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].bar(comparison_df['Model'], comparison_df['AUC-PR'], color='coral', alpha=0.7)
axes[0, 1].set_ylabel('AUC-PR')
axes[0, 1].set_title('AUC-PR Comparison')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Precision, Recall, F1
x = range(len(comparison_df))
width = 0.25
axes[1, 0].bar([i - width for i in x], comparison_df['Precision (Fraud)'], width, 
                label='Precision', color='green', alpha=0.7)
axes[1, 0].bar(x, comparison_df['Recall (Fraud)'], width, 
               label='Recall', color='blue', alpha=0.7)
axes[1, 0].bar([i + width for i in x], comparison_df['F1-Score (Fraud)'], width, 
               label='F1-Score', color='red', alpha=0.7)
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_title('Precision, Recall, F1-Score (Fraud Class)')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(comparison_df['Model'], rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Accuracy
axes[1, 1].bar(comparison_df['Model'], comparison_df['Accuracy'], color='purple', alpha=0.7)
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].set_title('Accuracy Comparison')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tìm best model
best_model_idx = comparison_df['F1-Score (Fraud)'].idxmax()
best_model = comparison_df.loc[best_model_idx, 'Model']
print(f"\n{'='*60}")
print(f"BEST MODEL: {best_model}")
print(f"{'='*60}")
print(f"F1-Score (Fraud): {comparison_df.loc[best_model_idx, 'F1-Score (Fraud)']:.4f}")
print(f"Recall (Fraud): {comparison_df.loc[best_model_idx, 'Recall (Fraud)']:.4f}")
print(f"Precision (Fraud): {comparison_df.loc[best_model_idx, 'Precision (Fraud)']:.4f}")
print(f"AUC-ROC: {comparison_df.loc[best_model_idx, 'AUC-ROC']:.4f}")
print(f"AUC-PR: {comparison_df.loc[best_model_idx, 'AUC-PR']:.4f}")


In [ ]:
### ANALYSE
import matplotlib.pyplot as plt

pdf_label = df_clean.select("TX_FRAUD").toPandas()

plt.figure()
pdf_label["TX_FRAUD"].value_counts().plot(kind="bar")
plt.title("Fraud vs Non-Fraud Distribution")
plt.xlabel("TX_FRAUD (0 = Normal, 1 = Fraud)")
plt.ylabel("Count")
plt.show()


In [ ]:
feature_cols = [
    "CUSTOMER_ID",
    "TERMINAL_ID",
    "TX_AMOUNT",
    "TX_TIME_SECONDS",
    "TX_TIME_DAYS",
    "TX_HOUR",
    "TX_DAYOFWEEK"
]


In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="keep"
)

df_ml = assembler.transform(df_clean) \
    .select("features", "TX_FRAUD")

df_ml.show(3, truncate=False)

In [ ]:
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

print("Train:", train_df.count())
print("Test :", test_df.count())


In [ ]:
### BASE LINE
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="TX_FRAUD",
    maxIter=20
)

lr_model = lr.fit(train_df)
lr_pred = lr_model.transform(test_df)


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="TX_FRAUD",
    metricName="areaUnderROC"
)

auc = evaluator.evaluate(lr_pred)
print("Logistic Regression AUC:", auc)


In [ ]:
lr_pred.groupBy("TX_FRAUD", "prediction").count().orderBy("TX_FRAUD", "prediction").show()


# Perform

Perform a comparative analysis of Logistic Regression and Random Forest models for fraud detection using the dataset `Final Transaction.csv`. The analysis should include:
1.  **Baseline Evaluation (Before SMOTE)**:
    *   Calculate the Recall score for the existing Logistic Regression model.
    *   Train a Random Forest Classifier and evaluate its AUC and Recall scores.
2.  **SMOTE Application**: Apply SMOTE to balance the training dataset.
3.  **Post-SMOTE Evaluation**:
    *   Train a Logistic Regression model on the SMOTE-balanced data and evaluate its AUC and Recall scores on the original unbalanced test set.
    *   Train a Random Forest model on the SMOTE-balanced data and evaluate its AUC and Recall scores on the original unbalanced test set.
4.  **Summary and Comparison**: Present a comprehensive summary of all AUC and Recall scores, comparing the performance of both models before and after SMOTE. Discuss the findings.

## Calculate Recall for Logistic Regression (Before SMOTE)

### Subtask:
Calculate and print the Recall score for the existing Logistic Regression model trained on the unbalanced dataset. We already have the predictions (`lr_pred`) and AUC.


**Reasoning**:
To calculate the recall score, I need to use `MulticlassClassificationEvaluator` from `pyspark.ml.evaluation`.



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_recall = MulticlassClassificationEvaluator(
    labelCol="TX_FRAUD",
    predictionCol="prediction",
    metricName="recallByLabel"
)

# Get recall for each class
recall_by_label = evaluator_recall.evaluate(lr_pred)

# Extract recall for the positive class (label 1)
# Note: evaluate returns a single aggregated metric. To get recall for specific label,
# one typically needs to either filter the predictions or configure the evaluator differently.
# However, the provided instruction `metricName='recallByLabel'` will return an aggregated recall.
# Let's adjust to get the recall for the positive class specifically.

# A common way to get per-class metrics is to manually calculate or use a confusion matrix
# or iterate through classes if the evaluator supports it (which it does via summary).
# Given the request, the simplest way to get recall for label 1 is to filter the data.

# For recall of label 1, we only care about actual label 1s that were predicted correctly
# True Positives (TX_FRAUD=1, prediction=1) / (True Positives + False Negatives)
# False Negatives (TX_FRAUD=1, prediction=0)

true_positives = lr_pred.filter((lr_pred.TX_FRAUD == 1) & (lr_pred.prediction == 1)).count()
false_negatives = lr_pred.filter((lr_pred.TX_FRAUD == 1) & (lr_pred.prediction == 0)).count()

recall_label_1 = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

print(f"Logistic Regression Recall for Class 1 (Fraud): {recall_label_1}")

## Train and Evaluate Random Forest (Before SMOTE)

### Subtask:
Train a Random Forest Classifier on the original unbalanced training dataset (`train_df`), make predictions on the test set (`test_df`), and then calculate both AUC and Recall scores for this model.


**Reasoning**:
The next step is to train a Random Forest Classifier, make predictions, and then calculate its AUC and Recall scores as per the subtask instructions.



In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Instantiate a RandomForestClassifier
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="TX_FRAUD",
    seed=42,
    numTrees=100 # Default numTrees is 20, using 100 for better performance
)

# 2. Fit the Random Forest model to the train_df DataFrame
rf_model = rf.fit(train_df)

# 3. Make predictions on the test_df DataFrame
rf_pred = rf_model.transform(test_df)

# Display some predictions
rf_pred.select("TX_FRAUD", "prediction", "probability").show(5)

# 4. Instantiate a BinaryClassificationEvaluator for AUC
auc_evaluator = BinaryClassificationEvaluator(
    labelCol="TX_FRAUD",
    metricName="areaUnderROC"
)

# 5. Evaluate the rf_pred DataFrame to calculate the AUC score
rf_auc = auc_evaluator.evaluate(rf_pred)
print(f"Random Forest AUC (Before SMOTE): {rf_auc}")

# 6. Calculate the Recall score for the fraud class (label 1)
rf_true_positives = rf_pred.filter((rf_pred.TX_FRAUD == 1) & (rf_pred.prediction == 1)).count()
rf_false_negatives = rf_pred.filter((rf_pred.TX_FRAUD == 1) & (rf_pred.prediction == 0)).count()

rf_recall_label_1 = rf_true_positives / (rf_true_positives + rf_false_negatives) if (rf_true_positives + rf_false_negatives) > 0 else 0

# 7. Print the calculated Recall score
print(f"Random Forest Recall for Class 1 (Fraud) (Before SMOTE): {rf_recall_label_1}")

# Show confusion matrix for RF model
rf_pred.groupBy("TX_FRAUD", "prediction").count().orderBy("TX_FRAUD", "prediction").show()

## Apply SMOTE to Training Data

### Subtask:
Convert the Spark training DataFrame (`train_df`) to a Pandas DataFrame, apply the SMOTE (Synthetic Minority Over-sampling Technique) algorithm to balance the classes, and then convert the balanced data back into a Spark DataFrame. This balanced dataset will be used for training subsequent models.


**Reasoning**:
The first step in applying SMOTE is to ensure the `imblearn` library is installed, which is not available by default in most environments. This needs to be done in a separate code block using `!pip install`.



In [ ]:
!pip install imblearn

**Reasoning**:
The `imblearn` library has been installed. Now, I will proceed with the remaining instructions of the subtask: converting the Spark DataFrame to Pandas, applying SMOTE, converting back to Spark, and verifying the balancing. All these steps are related to the SMOTE application and can be performed in one code block.



In [ ]:
# =========================
# SMOTE for Spark Training Data
# =========================

import pandas as pd
from imblearn.over_sampling import SMOTE
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import StructType, StructField, IntegerType

# -------------------------------------------------
# 1. Convert Spark DataFrame -> Pandas
# -------------------------------------------------
# train_df must contain: [features: VectorUDT, TX_FRAUD: int]

pd_train_df = train_df.select("features", "TX_FRAUD").toPandas()

# Convert Spark VectorUDT -> Pandas numerical matrix
X_pd = pd.DataFrame(
    pd_train_df["features"].apply(lambda v: v.toArray()).tolist()
)
y_pd = pd_train_df["TX_FRAUD"]

print("Original class distribution:")
print(y_pd.value_counts())

# -------------------------------------------------
# 2. Apply SMOTE
# -------------------------------------------------
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_pd, y_pd)

print("\nAfter SMOTE class distribution:")
print(pd.Series(y_res).value_counts())

# -------------------------------------------------
# 3. Convert back to Spark DataFrame
# -------------------------------------------------
data_for_spark = [
    (Vectors.dense(X_res.iloc[i].values.tolist()), int(y_res.iloc[i]))
    for i in range(len(y_res))
]

schema = StructType([
    StructField("features", VectorUDT(), True),
    StructField("TX_FRAUD", IntegerType(), True)
])

train_df_smote = spark.createDataFrame(data_for_spark, schema)

# -------------------------------------------------
# 4. Verification
# -------------------------------------------------
print("\nSpark SMOTE DataFrame:")
train_df_smote.printSchema()

print("Row count:", train_df_smote.count())
train_df_smote.groupBy("TX_FRAUD").count().show()


## Train Logistic Regression (After SMOTE)

### Subtask:
Train a new Logistic Regression model using the SMOTE-balanced training dataset created in the previous step.


**Reasoning**:
The subtask requires training a Logistic Regression model on the SMOTE-balanced data, which involves instantiating the model with specified parameters and then fitting it to the `train_df_smote` DataFrame.



In [ ]:
from pyspark.ml.classification import LogisticRegression

# 1. Instantiate a LogisticRegression model
lr_smote = LogisticRegression(
    featuresCol="features",
    labelCol="TX_FRAUD",
    maxIter=20
)

# 2. Fit this Logistic Regression model to the train_df_smote DataFrame
lr_model_smote = lr_smote.fit(train_df_smote)

print("Logistic Regression model trained on SMOTE-balanced data.")

## Evaluate Logistic Regression (After SMOTE)

### Subtask:
Make predictions on the original unbalanced `test_df` using the `lr_model_smote`, then calculate and print its AUC and Recall scores for the fraud class (label 1).

**Reasoning**:
To evaluate the Logistic Regression model trained on SMOTE-balanced data, I need to make predictions on the original test set and then compute the AUC and Recall scores, similar to how the baseline Logistic Regression model was evaluated.



In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Make predictions on the original test_df DataFrame using the lr_model_smote
lr_smote_pred = lr_model_smote.transform(test_df)

# Display some predictions
lr_smote_pred.select("TX_FRAUD", "prediction", "probability").show(5)

# 2. Instantiate a BinaryClassificationEvaluator for AUC
auc_evaluator = BinaryClassificationEvaluator(
    labelCol="TX_FRAUD",
    metricName="areaUnderROC"
)

# 3. Evaluate the lr_smote_pred DataFrame to calculate the AUC score
lr_smote_auc = auc_evaluator.evaluate(lr_smote_pred)
print(f"Logistic Regression AUC (After SMOTE): {lr_smote_auc}")

# 4. Calculate the Recall score for the fraud class (label 1)
lr_smote_true_positives = lr_smote_pred.filter((lr_smote_pred.TX_FRAUD == 1) & (lr_smote_pred.prediction == 1)).count()
lr_smote_false_negatives = lr_smote_pred.filter((lr_smote_pred.TX_FRAUD == 1) & (lr_smote_pred.prediction == 0)).count()

lr_smote_recall_label_1 = lr_smote_true_positives / (lr_smote_true_positives + lr_smote_false_negatives) if (lr_smote_true_positives + lr_smote_false_negatives) > 0 else 0

# 5. Print the calculated Recall score
print(f"Logistic Regression Recall for Class 1 (Fraud) (After SMOTE): {lr_smote_recall_label_1}")

# Show confusion matrix for LR model after SMOTE
lr_smote_pred.groupBy("TX_FRAUD", "prediction").count().orderBy("TX_FRAUD", "prediction").show()

## Train Random Forest (After SMOTE)

### Subtask:
Train a new Random Forest Classifier model using the SMOTE-balanced training dataset (`train_df_smote`) created earlier.

**Reasoning**:
Following the previous markdown block, the next logical step is to implement the training of the Random Forest Classifier on the SMOTE-balanced dataset. This involves instantiating the model with appropriate parameters and then fitting it to the `train_df_smote` DataFrame.



In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# 1. Instantiate a RandomForestClassifier
rf_smote = RandomForestClassifier(
    featuresCol="features",
    labelCol="TX_FRAUD",
    seed=42,
    numTrees=100 # Consistent with baseline RF model
)

# 2. Fit the Random Forest model to the train_df_smote DataFrame
rf_model_smote = rf_smote.fit(train_df_smote)

print("Random Forest model trained on SMOTE-balanced data.")

## Evaluate Random Forest (After SMOTE)

### Subtask:
Make predictions on the original unbalanced `test_df` using the `rf_model_smote`, then calculate and print its AUC and Recall scores for the fraud class (label 1).

**Reasoning**:
To evaluate the Random Forest model trained on SMOTE-balanced data, I need to make predictions on the original test set and then compute the AUC and Recall scores, similar to how the previous models were evaluated.



In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Make predictions on the original test_df DataFrame using the rf_model_smote
rf_smote_pred = rf_model_smote.transform(test_df)

# Display some predictions
rf_smote_pred.select("TX_FRAUD", "prediction", "probability").show(5)

# 2. Instantiate a BinaryClassificationEvaluator for AUC
auc_evaluator = BinaryClassificationEvaluator(
    labelCol="TX_FRAUD",
    metricName="areaUnderROC"
)

# 3. Evaluate the rf_smote_pred DataFrame to calculate the AUC score
rf_smote_auc = auc_evaluator.evaluate(rf_smote_pred)
print(f"Random Forest AUC (After SMOTE): {rf_smote_auc}")

# 4. Calculate the Recall score for the fraud class (label 1)
rf_smote_true_positives = rf_smote_pred.filter((rf_smote_pred.TX_FRAUD == 1) & (rf_smote_pred.prediction == 1)).count()
rf_smote_false_negatives = rf_smote_pred.filter((rf_smote_pred.TX_FRAUD == 1) & (rf_smote_pred.prediction == 0)).count()

rf_smote_recall_label_1 = rf_smote_true_positives / (rf_smote_true_positives + rf_smote_false_negatives) if (rf_smote_true_positives + rf_smote_false_negatives) > 0 else 0

# 5. Print the calculated Recall score
print(f"Random Forest Recall for Class 1 (Fraud) (After SMOTE): {rf_smote_recall_label_1}")

# Show confusion matrix for RF model after SMOTE
rf_smote_pred.groupBy("TX_FRAUD", "prediction").count().orderBy("TX_FRAUD", "prediction").show()

## Summary and Comparison

### Subtask:
Present a comprehensive summary of all AUC and Recall scores, comparing the performance of both models before and after SMOTE. Discuss the findings.

### Instructions:
1. Create a summary table or dictionary of all calculated AUC and Recall scores for:
    *   Logistic Regression (Before SMOTE)
    *   Random Forest (Before SMOTE)
    *   Logistic Regression (After SMOTE)
    *   Random Forest (After SMOTE)
2. Print this summary.
3. Discuss the findings based on the comparison of these metrics.

**Reasoning**:
All models have been trained and evaluated, both before and after applying SMOTE. The next step is to create a summary of all calculated AUC and Recall scores and then provide a discussion of the findings, as specified in the subtask.



In [ ]:
print("### Summary of Model Performance\n")

results = {
    "Logistic Regression (Before SMOTE)": {
        "AUC": auc,
        "Recall": recall_label_1
    },
    "Random Forest (Before SMOTE)": {
        "AUC": rf_auc,
        "Recall": rf_recall_label_1
    },
    "Logistic Regression (After SMOTE)": {
        "AUC": lr_smote_auc,
        "Recall": lr_smote_recall_label_1
    },
    "Random Forest (After SMOTE)": {
        "AUC": rf_smote_auc,
        "Recall": rf_smote_recall_label_1
    }
}

for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  AUC: {metrics['AUC']:.4f}")
    print(f"  Recall (Fraud Class): {metrics['Recall']:.4f}\n")



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd

# Function to get probabilities and true labels from Spark DataFrame
def get_roc_data(spark_df, label_col="TX_FRAUD", prob_col="probability"):
    pdf = spark_df.select(label_col, prob_col).toPandas()
    y_true = pdf[label_col].values
    # Extract probability of the positive class (label 1)
    y_proba = pdf[prob_col].apply(lambda x: x[1]).values
    return y_true, y_proba

# Prepare data for plotting ROC curves
lr_y_true, lr_y_proba = get_roc_data(lr_pred)
rf_y_true, rf_y_proba = get_roc_data(rf_pred)
lr_smote_y_true, lr_smote_y_proba = get_roc_data(lr_smote_pred)
rf_smote_y_true, rf_smote_y_proba = get_roc_data(rf_smote_pred)

# Calculate ROC curve and AUC for each model
lr_fpr, lr_tpr, _ = roc_curve(lr_y_true, lr_y_proba)
lr_auc = auc(lr_fpr, lr_tpr)

rf_fpr, rf_tpr, _ = roc_curve(rf_y_true, rf_y_proba)
rf_auc_val = auc(rf_fpr, rf_tpr) # Renamed to avoid conflict with existing rf_auc variable

lr_smote_fpr, lr_smote_tpr, _ = roc_curve(lr_smote_y_true, lr_smote_y_proba)
lr_smote_auc_val = auc(lr_smote_fpr, lr_smote_tpr)

rf_smote_fpr, rf_smote_tpr, _ = roc_curve(rf_smote_y_true, rf_smote_y_proba)
rf_smote_auc_val = auc(rf_smote_fpr, rf_smote_tpr)

# Plotting the ROC curves
plt.figure(figsize=(10, 8))
plt.plot(lr_fpr, lr_tpr, color='blue', lw=2, label=f'LR (Before SMOTE) AUC = {lr_auc:.2f}')
plt.plot(rf_fpr, rf_tpr, color='green', lw=2, label=f'RF (Before SMOTE) AUC = {rf_auc_val:.2f}')
plt.plot(lr_smote_fpr, lr_smote_tpr, color='red', lw=2, label=f'LR (After SMOTE) AUC = {lr_smote_auc_val:.2f}')
plt.plot(rf_smote_fpr, rf_smote_tpr, color='purple', lw=2, label=f'RF (After SMOTE) AUC = {rf_smote_auc_val:.2f}')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve Comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Summary:

### Data Analysis Key Findings

*   **Logistic Regression (Before SMOTE)**: Achieved an AUC of 0.9914 and a Recall for the fraud class of 0.9758.
*   **Random Forest (Before SMOTE)**: Achieved an AUC of 0.9888 and a Recall for the fraud class of 0.9426.
*   **Impact of SMOTE on Training Data**: The SMOTE algorithm successfully balanced the training dataset, resulting in an equal number of instances (168,763) for both fraud and non-fraud classes.
*   **Logistic Regression (After SMOTE)**:
    *   The AUC remained high at 0.9914, showing consistent overall discrimination.
    *   The Recall for the fraud class improved to 0.9835, indicating better detection of fraudulent transactions after balancing the training data.
*   **Random Forest (After SMOTE)**:
    *   The AUC increased slightly to 0.9914, demonstrating improved overall performance.
    *   The Recall for the fraud class significantly improved to 0.9750, addressing its lower baseline recall.
*   **Post-SMOTE Model Comparison**: After applying SMOTE, both Logistic Regression and Random Forest models demonstrated excellent and very similar AUC scores (0.9914). Logistic Regression maintained a slight edge in Recall for the fraud class (0.9835) compared to Random Forest (0.9750).

### Insights or Next Steps

*   **SMOTE's Effectiveness**: Applying SMOTE significantly improved the recall for the minority fraud class in both Logistic Regression (from 0.9758 to 0.9835) and Random Forest (from 0.9426 to 0.9750) models, making it a valuable strategy for handling imbalanced datasets in fraud detection.
*   **Model Recommendation**: Given the critical importance of minimizing false negatives in fraud detection, the Logistic Regression model trained with SMOTE-balanced data (Recall: 0.9835) appears to be the marginally preferred choice over Random Forest with SMOTE (Recall: 0.9750) in this scenario.
